# IIJA All Projects 

In [1]:
# ! pip install nltk
# ! pip install textblob

In [2]:
import numpy as np
import pandas as pd
from siuba import *

from shared_utils import geography_utils
from dla_utils import _dla_utils

from calitp import to_snakecase

import utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [3]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)


## Read Data

In [4]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [5]:
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/CopyofFMIS_Projects_Universe_IIJA_Reporting_4.xls", 
                           # sheet_name='FMIS 5 Projects  ', header=[3]
                           sheet_name='IIJA',
                           # sheet_name='FMIS 5 Projects  ',
                           ))

In [6]:
proj.drop(columns =['unnamed:_0', 'unnamed:_13', 'unnamed:_14'], axis=1, inplace=True)

In [7]:
proj.sample(5)

fmis_transaction_date program_code      program_code_description  \
206             2022-03-03         Y233   STBG IIJA OFF-SYSTEM BRIDGE   
100             2022-02-10         YS30  HIGHWAY SAFETY IMP PROG IIJA   
1218            2022-08-31         Y001    NATIONAL HIGHWAY PERF IIJA   
60              2022-07-29         Y230     STBG-URBANIZED >200K IIJA   
793             2022-09-19         YS30  HIGHWAY SAFETY IMP PROG IIJA   

     project_number recipient_project_number  \
206         5182057              0300020208L   
100         5043043              0419000276L   
1218        2101855              0716000061S   
60          5006929              0723000002L   
793         P070144              0321000092S   

                                                                                                                                              project_title  \
206                                                                                       OAK RIDGE DRIVE AT LINDA CREEK BR#19C0180 BRIDGE REPLACEMENT (TC)   
100           THIRD STREET AT THE THE INTERSECTIONS OF LINDARO STREET, LINCOLN AVENUE, TAMALPAIS AVENUE, IRWIN STREET AND GRAND AVENUE. SAFETY IMPROVEMENTS   
1218  IN LOS ANGELES COUNTY IN IRWINDALE AND AZUSA FROM 0.1 MILE EAST OF HIGHLAND AVENUE UNDERCROSSING TO SOUTH CITRUS AVENUE OVERCROSSING. CONSTRUCT AND I   
60    VARIOUS  LOCAL STREETS WITHIN THE CITY OF LOS ANGELES TO RE-SURFACE  LOCAL STREETS OF APPROXIMATELY 45 MILE IN TOTAL LENGTH  AND CONSTRUCT ACCESS RAM   
793         YUBA COUNTY NEAR MARYSVILLE FROM 0.2 MILE NORTH OF LAURELLEN ROAD TO SOUTH HONCUT CREEK BRIDGE WIDEN SHOULDERS AND IMPROVE CLEAR RECOVERY ZONE.   

      county_code congressional_district  improvement_type  \
206         61.00            Cong Dist 4             43.00   
100         41.00            Cong Dist 2             21.00   
1218        37.00           Cong Dist 32             37.00   
60          37.00           Cong Dist 37              5.00   
793        115.00            Cong Dist 3             42.00   

                             improvement_type_description  obligations_amount  \
206                                             Utilities           50,000.00   
100                                                Safety        1,277,800.00   
1218  Mitigation of Water Pollution due to Highway Runoff        3,151,100.00   
60                          4R - Maintenance  Resurfacing        1,955,000.00   
793                                              Training            6,100.00   

     summary_recipient_defined_text_field_1_value  
206                                    L5182SACOG  
100                                      L5043MTC  
1218                                    S    SCAG  
60                                      L5006SCAG  
793                                    S    SACOG

In [8]:
proj = proj.dropna(how='all') 

In [9]:
proj['summary_recipient_defined_text_field_1_value'] = proj['summary_recipient_defined_text_field_1_value'].fillna(value='None')

In [10]:
proj.head(1)

fmis_transaction_date program_code   program_code_description  \
0            2022-03-03         Y230  STBG-URBANIZED >200K IIJA   

  project_number recipient_project_number  \
0        5002174              0314000222L   

                                                             project_title  \
0  BELL AVE. FROM BOLLENBACHER AVE. TO ASTORIA ST ROAD REHABILITATION (TC)   

   county_code congressional_district  improvement_type  \
0        67.00            Cong Dist 6              4.00   

  improvement_type_description  obligations_amount  \
0       4R - No Added Capacity        2,841,397.00   

  summary_recipient_defined_text_field_1_value  
0                                   L5002SACOG

In [11]:
len(proj)

1241

In [12]:
## agency identifier column doesnt always have locode

## Function Work

In [13]:
# (proj[proj['summary_recipient_defined_text_field_1_value'].str.startswith('L')]).sample()

In [14]:
# (proj[proj['summary_recipient_defined_text_field_1_value'].str.startswith('L')]).sample()

In [15]:
#seeing if the space is the way to filter
# ((proj[~proj['summary_recipient_defined_text_field_1_value'].str.contains(" ")])>>count(_.summary_recipient_defined_text_field_1_value))

In [16]:
#check to make sure there are numbers
# ((proj[~proj['summary_recipient_defined_text_field_1_value'].str.contains(" ")])>>count(_.summary_recipient_defined_text_field_1_value)).iloc[1:20]

In [17]:
# locode_proj = ((proj[~proj['summary_recipient_defined_text_field_1_value'].str.contains(" ")]))
# locode_proj = locode_proj>>filter(_.summary_recipient_defined_text_field_1_value!='None')

In [18]:
# len(locode_proj)

In [19]:
# locode_proj = (utils.add_name_from_locode(locode_proj, 'summary_recipient_defined_text_field_1_value'))

In [20]:
# no_locode = ((proj[proj['summary_recipient_defined_text_field_1_value'].str.contains(" ")]))

In [21]:
# no_entry = proj>>filter(_['summary_recipient_defined_text_field_1_value']=='None')
    
#     #concat no locodes and those with no entry
# no_locode = pd.concat([no_locode, no_entry])

In [22]:
# no_locode.sample()

In [23]:
# no_locode = no_locode.rename(columns = {"recipient_name":"implementing_agency", "county_description":"county_name"})

In [24]:
# len(no_locode)

In [25]:
# pd.concat([locode_proj, no_locode])

## Function for adding Implemeting Agency Information

In [26]:
# function moved to utils

In [27]:
df = utils.identify_agency(proj, 'summary_recipient_defined_text_field_1_value')

In [28]:
df.sample()

fmis_transaction_date program_code      program_code_description  \
98            2022-08-29         YS30  HIGHWAY SAFETY IMP PROG IIJA   

   project_number recipient_project_number  \
98        P020199              0117000018S   

                                                                                                                                            project_title  \
98  IN LAKE COUNTY NEAR CLEAR LAKE OAKS AT BERYL WAY THIS PROJECT PROPOSES TO INSTALL A COMBINATION OF TWO ALTERNATIVE IN-LINE TERMINAL SYSTEMS, A RAIL E   

    county_code congressional_district  improvement_type  \
98        33.00            Cong Dist 3             17.00   

   improvement_type_description  obligations_amount  \
98     Construction Engineering          434,237.51   

   summary_recipient_defined_text_field_1_value  implementing_agency_locode  \
98                                 S    NON-MPO                         NaN   

   implementing_agency  district  county_name rtpa_name mpo_name  
98          California      1.00  Lake County       NaN      NaN

In [29]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1254 entries, 0 to 671
Data columns (total 18 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   fmis_transaction_date                         1254 non-null   datetime64[ns]
 1   program_code                                  1254 non-null   object        
 2   program_code_description                      1254 non-null   object        
 3   project_number                                1254 non-null   object        
 4   recipient_project_number                      1253 non-null   object        
 5   project_title                                 1254 non-null   object        
 6   county_code                                   1254 non-null   float64       
 7   congressional_district                        1254 non-null   object        
 8   improvement_type                              1254 non-null   float64

In [30]:
df = (utils.add_description(df, 'project_title'))

In [31]:
df>>filter(_.project_type=="Project")

fmis_transaction_date program_code      program_code_description  \
2              2022-08-26         Y400    CONGESTION MITIGATION IIJA   
3              2022-08-26         Y400    CONGESTION MITIGATION IIJA   
4              2022-07-15         Y400    CONGESTION MITIGATION IIJA   
5              2022-02-08         YS30  HIGHWAY SAFETY IMP PROG IIJA   
6              2022-02-08         YS30  HIGHWAY SAFETY IMP PROG IIJA   
..                    ...          ...                           ...   
662            2022-06-16         Y001    NATIONAL HIGHWAY PERF IIJA   
663            2022-06-16         Y001    NATIONAL HIGHWAY PERF IIJA   
664            2022-06-16         Y001    NATIONAL HIGHWAY PERF IIJA   
665            2022-06-16         Y001    NATIONAL HIGHWAY PERF IIJA   
671            2022-09-23         YR10   STATE REC TRAILS ADMIN IIJA   

    project_number recipient_project_number  \
2          5002178              0315000001L   
3          5002178              0315000001L   
4          5002189              0316000158L   
5          5002199              0317000138L   
6          5002199              0317000138L   
..             ...                      ...   
662        4053001              0716000031S   
663        4053001              0716000031S   
664        4053001              0716000031S   
665        4053001              0716000031S   
671        2023000                      NaN   

                                                                                                                                             project_title  \
2    SOUTH SACRAMENTO PARKWAY TRAIL- WEST. IN THE CITY OF SACRAMENTO, ADJACENT TO NORTHBOUND INTERSTATE 5, SOUTH OF POCKET ROAD/MEADOWVIEW ROAD AND FREEPO   
3    SOUTH SACRAMENTO PARKWAY TRAIL- WEST. IN THE CITY OF SACRAMENTO, ADJACENT TO NORTHBOUND INTERSTATE 5, SOUTH OF POCKET ROAD/MEADOWVIEW ROAD AND FREEPO   
4    ALONG UNUSED RAIL CORRIDOR NEAR FREEPORT BLVD FROM SUTTERVILLE ROAD TO SOUTH OF MEADOWVIEW/POCKET ROAD (4.5 MILES) RAILS TO TRAILS PROJECT ; NEW CLAS   
5    INTERSECTIONS OF 65TH ST EXPWY AT 21ST AVE, 65TH ST EXPWY AT FRUITRIDGE RD, ARDEN WAY AT HERITAGE LN, ARDEN WAY AT CHALLENGE WAY, HOWE AVE. AT SWARTH   
6    INTERSECTIONS OF 65TH ST EXPWY AT 21ST AVE, 65TH ST EXPWY AT FRUITRIDGE RD, ARDEN WAY AT HERITAGE LN, ARDEN WAY AT CHALLENGE WAY, HOWE AVE. AT SWARTH   
..                                                                                                                                                     ...   
662  IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE   
663  IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE   
664  IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE   
665  IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE   
671                                                                                  2023 REC TRAILS PROJECT: STATE PARKS INCURRED ADMINISTRATIVE EXPENSES   

     county_code congressional_district  improvement_type  \
2          67.00            Cong Dist 6             17.00   
3          67.00            Cong Dist 6             28.00   
4          67.00            Cong Dist 6             28.00   
5          67.00            Cong Dist 3             21.00   
6          67.00            Cong Dist 6             17.00   
..           ...                    ...               ...   
662        37.00           Cong Dist 29             44.00   
663        37.00           Cong Dist 30             17.00   
664        37.00           Cong Dist 30             42.00   
665        37.00           Cong Dist 30             44.00   
671   

In [32]:
df.sample()

fmis_transaction_date program_code    program_code_description  \
314            2022-07-12         Y001  NATIONAL HIGHWAY PERF IIJA   

    project_number recipient_project_number  \
314        5904156              0116000098L   

                                                                                               project_title  \
314  VARIOUS LOCATIONS IN HUMBOLDT COUNTY BRIDGE PREVENTIVE MAINTENANCE: MINOR CONCRETE REPAIR ON 8 BRIDGES.   

     county_code congressional_district  improvement_type  \
314        23.00            Cong Dist 2             17.00   

    improvement_type_description  obligations_amount  \
314     Construction Engineering           75,493.96   

    summary_recipient_defined_text_field_1_value  implementing_agency_locode  \
314                                 L5904NON-MPO                    5,904.00   

    implementing_agency  district      county_name  \
314     Humboldt County      1.00  Humboldt County   

                                      rtpa_name mpo_name project_method  \
314  Humboldt County Association of Governments  NON-MPO         Repair   

                      project_type  
314  Bridge Preventive Maintenance

In [33]:
df.loc[df.county_name == "Statewide County", 'county_name'] = "Statewide"

## New Title Function

In [34]:
def add_new_title(df, first_col_method, second_col_type, third_col_name, alt_col_name):
    """
    Function to add new title. 
    Expected output example: "New Bike Lane in Eureka"
    """
    def return_name(df):
        
        if (df[third_col_name] == "California") & (df[alt_col_name] == "Statewide"):
            return (df[first_col_method] + " " + df[second_col_type] +" " + df[alt_col_name])
        
        elif (df[third_col_name] == "California"):
            return (df[first_col_method] + " " + df[second_col_type] + " in " + df[alt_col_name])
        
        elif (df[third_col_name] != "California"):
            return (df[first_col_method] + " " + df[second_col_type] + " in " + df[third_col_name])
        
        # elif (df[third_col_name] == "Metropolitan Transportation Commission"):
        #     return (df[first_col_method] + " " + df[second_col_type] + " in The " + df[third_col_name])

        return df

    df['project_name_new'] = df.apply(return_name, axis = 1)
    
    return df

In [35]:
df = add_new_title(df, "project_method", "project_type", "implementing_agency", "county_name")

In [36]:
df.tail()


fmis_transaction_date program_code      program_code_description  \
667            2022-09-09         YS30  HIGHWAY SAFETY IMP PROG IIJA   
668            2022-09-09         YS30  HIGHWAY SAFETY IMP PROG IIJA   
669            2022-09-09         YS30  HIGHWAY SAFETY IMP PROG IIJA   
670            2022-09-09         YS32   SEC 164 PENALTIES HSIP IIJA   
671            2022-09-23         YR10   STATE REC TRAILS ADMIN IIJA   

    project_number recipient_project_number  \
667        5801081              0416000125S   
668        5801081              0416000125S   
669        5801081              0416000125S   
670        5801081              0416000125S   
671        2023000                      NaN   

                                                                                                                     project_title  \
667  ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM   
668  ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM   
669  ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM   
670  ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM   
671                                                          2023 REC TRAILS PROJECT: STATE PARKS INCURRED ADMINISTRATIVE EXPENSES   

     county_code congressional_district  improvement_type  \
667         1.00           Cong Dist 15             21.00   
668         1.00           Cong Dist 15             22.00   
669         1.00           Cong Dist 15             42.00   
670         1.00           Cong Dist 15             17.00   
671       999.00            Cong Dist 0             44.00   

    improvement_type_description  obligations_amount  \
667                       Safety        4,165,800.00   
668            Rail/Hwy Crossing           18,900.00   
669                     Training            3,000.00   
670     Construction Engineering          288,000.00   
671                        Other          241,760.00   

    summary_recipient_defined_text_field_1_value  implementing_agency_locode  \
667                                     S    MTC                         NaN   
668                                     S    MTC                         NaN   
669                                     S    MTC                         NaN   
670                                     S    MTC                         NaN   
671                                         None                         NaN   

    implementing_agency  district     county_name rtpa_name mpo_name  \
667          California      4.00  Alameda County       NaN      NaN   
668          California      4.00  Alameda County       NaN      NaN   
669          California      4.00  Alameda County       NaN      NaN   
670          California      4.00  Alameda County       NaN      NaN   
671          California       NaN       Statewide       NaN      NaN   

    project_method project_type                    project_name_new  
667        Install     Lighting  Install Lighting in Alameda County  
668        Install     Lighting  Install Lighting in Alameda County  
669        Install     Lighting  Install Lighting in Alameda County  
670        Install     Lighting  Install Lighting in Alameda County  
671                     Project                   Project Statewide

In [37]:
df>>filter(_.implementing_agency==("Metropolitan Transportation Commission"))>>select(_.project_name_new)

project_name_new
518                   Bike Share Program in Metropolitan Transportation Commission
519                      Mapping Project in Metropolitan Transportation Commission
520                              Signage in Metropolitan Transportation Commission
521                              Project in Metropolitan Transportation Commission
522                              Project in Metropolitan Transportation Commission
523                              Project in Metropolitan Transportation Commission
524         Regional Planning Activities in Metropolitan Transportation Commission
525         Regional Planning Activities in Metropolitan Transportation Commission
526         Regional Planning Activities in Metropolitan Transportation Commission
527         Regional Planning Activities in Metropolitan Transportation Commission
528         Regional Planning Activities in Metropolitan Transportation Commission
529         Regional Planning Activities in Metropolitan Transportation Commission
530         Regional Planning Activities in Metropolitan Transportation Commission
531         Regional Planning Activities in Metropolitan Transportation Commission
532         Regional Planning Activities in Metropolitan Transportation Commission
533         Regional Planning Activities in Metropolitan Transportation Commission
534         Regional Planning Activities in Metropolitan Transportation Commission
535         Regional Planning Activities in Metropolitan Transportation Commission
536  Implement Streets and Roads Program in Metropolitan Transportation Commission
537  Implement Streets and Roads Program in Metropolitan Transportation Commission
538  Implement Streets and Roads Program in Metropolitan Transportation Commission

### Export progress

In [38]:
#test_df = utils.title_column_names(df)

In [39]:
#test_df.to_csv(f"{GCS_FILE_PATH}/FMIS_projects_wip_all.csv")

## Use Alt Column

In [40]:
df.sample()

fmis_transaction_date program_code     program_code_description  \
18            2022-05-05         YS32  SEC 164 PENALTIES HSIP IIJA   

   project_number recipient_project_number  \
18        000C524              0414000003S   

                                                                          project_title  \
18  ALAMEDA AND CONTRA COSTA COUNTIES AT VARIOUS LOCATIONS CROSSWALK SAFETY ENHANCEMENT   

    county_code congressional_district  improvement_type  \
18       999.00            Cong Dist 0             42.00   

   improvement_type_description  obligations_amount  \
18                     Training            2,600.00   

   summary_recipient_defined_text_field_1_value  implementing_agency_locode  \
18                                     S    MTC                         NaN   

   implementing_agency  district county_name rtpa_name mpo_name  \
18          California       NaN   Statewide       NaN      NaN   

   project_method project_type    project_name_new  
18                     Project   Project Statewide

In [41]:
# using those with no match

no_match = (df>>filter(_.project_type==('Project'))>>select(_.project_number, _.recipient_project_number, _.project_title, _.improvement_type_description, _.implementing_agency, _.county_name,
                                                            _.project_method, _.project_type, _.project_name_new))

In [42]:
no_match.improvement_type_description.value_counts()

Construction Engineering                               179
Safety                                                  74
Training                                                69
4R - Restoration & Rehabilitation                       52
Planning                                                44
Preliminary Engineering                                 38
4R - Maintenance  Resurfacing                           19
Facilities for Pedestrians and Bicycles                 18
Bridge Replacement - No Added Capacity                  14
Right of Way                                            13
4R - No Added Capacity                                  12
Other                                                    9
Bridge Rehabilitation - No Added Capacity                6
4R - Added Capacity                                      6
Mitigation of Water Pollution due to Highway Runoff      4
New  Construction Roadway                                4
Bridge Replacement - Added Capacity                     

In [43]:
no_match>>filter(_.improvement_type_description=="Preliminary Engineering")

project_number recipient_project_number  \
135        5065024              0812000141L   
149        5085061              1022000247L   
158        5104048              0822000093L   
231        5213058              0417000332L   
256        5288043              0318000300L   
294        5447055              0323000032L   
298        5451035              1220000061L   
303        5478017              1222000033L   
307        5901044              0100020389L   
383        5925183              0321000054L   
388        5927131              0422000065L   
424        5937205              0416000438L   
426        5937206              0417000076L   
428        5937207              0417000077L   
448        5941100              0614000226L   
452        5941104              0615000093L   
454        5942198              0600020576L   
460        5942287              0618000119L   
472        5946106              0600020683L   
473        5946114              0600020688L   
479        5949120                05930239L   
516        6071169              1222000101L   
517        6071169              1222000101L   
522        6084282              0422000469L   
523        6084282              0422000469L   
545        6204134              0419000044L   
573        NBIL536              0815000049L   
47         00PE023              00000PE023S   
48         00PE023              00000PE023S   
49         00PE023              00000PE023S   
140        P051031              0316000113S   
283        P140035              1013000243S   
327        P905024              1119000045S   
463        X081034              0419000050S   
515        0051994              1121000129S   
516        0051994              1121000129S   
517        0051994              1121000129S   
518        0051994              1121000129S   

                                                                                                                                             project_title  \
135  BARTON ROAD OVERHEAD AT UNION PACIFIC RAILROAD, 0.25 MILE WEST OF I-215, BR. NO. 54C-0379 REPLACE OVERHEAD WITH TWO-LANE ROAD ACROSS DISCONTINUED RAI   
149  NORTH SIDE OF CHILDS AVE (FROM GOLDEN VALLEY HIGH SCHOOL TO COFFEE ST. (WEAVER MIDDLE SCHOOL)). CONSTRUCT A CLASS I (FULLY-SEPARATED). "SAFE ROUTES T   
158  PM00222/PSTUDY002 ON VARIOUS BRIDGE LOCATIONS THROUGHOUT THE CITY OF CORONA.  SEE COMMENTS FOR THE LIST OF BRIDGES. DEVELOP A BPMP PLAN FOR THE CITY-   
231                                                                                  VARIOUS LOCATION IN SUNNYVALE PROVIDE ADVANCE DILEMMA ZONE DETECTION.   
256                                                                                           US-50 AT EMPIRE RANCH ROAD CONSTRUCT 4 LANE INTERCHANGE (TC)   
294  PROJECT SPANS SACRAMENTO RIVER FROM 2ND/D STREET IN WEST SACRAMENTO, UP TO THE TOP DECK OF THE I STREET BRIDGE TO THE JIBBOOM/I STREET VIADUCT AND IN   
298  CITY OF MISSION VIEJO - THE PROJECT IS LOCATED ON THE WESTERN EDGE OF MISSION VIEJO. THE MISSION VIEJO N/S CORRIDOR TRAIL IS BOUNDED BY LOS ALISOS BL   
303  E/B SANTA MARGARITA PARKWAY OVER ARROYO TRABUCO (BRIDGE #55C0520R) AND ANTONIO PARKWAY OVER TIJERAS CREEK (BRIDGE #55C0605), BPMP PM00035 BRIDGE PREV   
307                                                               REQUA ROAD OVER HUNTER CREEK, 0.1 MI. WEST OF SR-101 REPLACE EXISTING 2-LANE BRIDGE (TC)   
383  IN EASTERN EI DORADO COUNTY, IN THE TAHOE BASIN, IN THE CITY OF SOUTH LAKE TAHOE AT EAST END OF SOUTH AVENUE, OVER THE UPPER TRUCKEE RIVER, THEN NORT   
388  4 AT TERNERS DRIVE @ DRIVEWAY ENTRANCES TO MULTI-FAMILY HOUSING CLOSEST TO DONAHUE, 4 AT TERNERS DRIVE @ TERRACE WAY, 4 AT TERNERS DRIVE @ TERRACE DR   
424  ALDER CROFT HEIGHTS ROAD OVER HOOKER CREEK, 2.12 MILES SOUTH OF LOS GATOS SC ROAD.   BRIDGE NO. 37C0506 REPLACE EXISTING ONE LANE BRIDGE WITH A NEW T   
426  ALDERCROFT HEIGHTS ROAD OVER LOS GATOS CREEK, 2.12 MILES S OF LOS GATOS CREEL ROAD. BRIDGE 37C0507 REPLACE EXIS

In [44]:
((utils.get_list_of_words((no_match>>filter(_.improvement_type_description=="4R - Restoration & Rehabilitation")), "project_title")).value_counts()).iloc[0:20]

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


mile             40
route            29
county           29
san              21
near             19
south            18
road             16
north            14
east             13
street           13
construct        13
undercrossing    13
05               13
various          11
diego            11
02               11
locations        11
los              11
separation       11
west             10
dtype: int64

In [45]:
no_match>>filter(_.project_title.str.contains("BICYC"), _.improvement_type_description!="Facilities for Pedestrians and Bicycles")

Empty DataFrame
Columns: [project_number, recipient_project_number, project_title, improvement_type_description, implementing_agency, county_name, project_method, project_type, project_name_new]
Index: []

In [46]:
(no_match>>filter(_.improvement_type_description=="4R - Restoration & Rehabilitation"))

project_number recipient_project_number  \
79         5012157              0419000338L   
183        5152022              0422000001L   
232        5213060              0418000456L   
247        5268022              0421000026L   
281        5379025              0422000039L   
299        5456021              1021000206L   
579        X099645              1000000424S   
1          6801946              0414000019S   
4          8051641              1117000037S   
6          8051641              1117000037S   
12         8801912              0417000035S   
29         000C533              0417000023S   
39         000C536              0416000036S   
50         19A5004              0512000009S   
71         P014107              0717000157S   
79         P017117              0414000404S   
88         P020196              0319000021S   
141        P051032              0317000064S   
218        P082031              0416000124S   
257        P118068              0716000083S   
260        P118068              0716000083S   
263        P118068              0716000083S   
271        P125020              1112000143S   
312        P241009              1212000053S   
337        Q101392              0116000137S   
345        Q101396              0517000001S   
348        Q101396              0517000001S   
351        Q101397              0416000017S   
366        S108057              1013000266S   
392        X019032              0616000234S   
404        X037211              0714000009S   
407        X037211              0714000009S   
410        X037211              0714000009S   
450        X073145              1117000038S   
452        X073148              1112000155S   
464        X093010              0217000009S   
540        0053074              0716000064S   
549        0055155              0615000046S   
580        0081328              1119000177S   
583        0081328              1119000177S   
586        0081329              1116000059S   
603        0154210              1119000008S   
618        0804211              0320000036S   
620        0804212              0321000109S   
622        0804212              0321000109S   
625        0804213              0321000106S   
628        1101849              0716000401S   
630        1101849              0716000401S   
632        1101849              0716000401S   
634        2101853              0716000217S   
651        2801153              0416000024S   
654        2801153              0416000024S   

                                                                                                                                             project_title  \
79   OAKPORT ST FROM EDGEWATER TO I-880 FREEWAY ENTRANCE; OAKLAND AVE FROM ORANGE STREET TO MACARTHUR; MONTEREY BLVD FROM MAIDEN LN TO BENNET PL IN OAKLAN   
183  EAST DUNNE AVENUE BETWEEN LOWER THOMAS GRADE AND THE EASTERN CITY LIMIT. FEDERAL PARTICIPATING LOCATION SEGMENTS: 1) LOWER THOMAS GRADE TO FLAMING OA   
232  IN EAST SUNNYVALE SENSE OF PLACE PLAN AREA: ON DUANE AVENUE FROM FAIR OAKS AVENUE TO LAWRENCE EXPRESSWAY, STEWART DRIVE FROM WOLFE ROAD TO DUANE AVEN   
247  BELMONT: CHULA VISTA FROM ALAMEDA DE LAS PULGAS TO RALSTON AVE, 6TH AVENUE FROM RALSTON AVENUE TO HILL STREET, 6TH AVENUE FROM EMMETT AVENUE TO HARBO   
281  COLEMAN CREEK PATH @ SNYDER LANE, HINEBAUGH CREEK PATH @ COUNTRY CLUB DRIVE, HINEBAUGH CREEK PATH @ STATE FARM DRIVE, HINEBAUGH CREEK PATH @ COMMERCE   
299  GOLDEN VALLEY PARKWAY (TOWNE CENTRE DRIVE AND LATHROP ROAD), RIVER ISLANDS PARKWAY (GOLDEN VALLEY PARKWAY LATHROP FIRE STATION 34), LATHROP ROAD. ROA   
579   IN AND NEAR THE CITY OF MODESTO ON SR 132 NEAR NORTH DAKOTA AVE TO NEAR 6TH ST AND ON SR 99 FROM 0.5 M SOUTH OF TUOLUMNE RIVER BRIDGE TO 0.1 M NORTH   
1                                                                   ALAMEDA COUNTY AT SUNOL AND IN PLEASANTON AND DUBLIN PRECAST CONCRETE PAVEMENT AND HMA   
4    IN SAN DIEGO COUNTY IN CHULA VISTA, FROM 0.3 MILE SOUTH OF MAIN STREET UNDERCROSSING 

### Function for no_matches

In [47]:
# def update_no_matched(df, flag_col, desc_col, name_col):
#     """
#     function to itreate over projects that did not match the first time
#     using an existing project's short description of project type. 
#     """
    
#     def return_project_type(df):
        
#         if (df[flag_col] == "Project") & (df[desc_col] == "Bridge Rehabilitation") | (df[desc_col] =="Bridge Rehabilitation - No Added Capacity") | (df[desc_col] =="Bridge Rehabilitation - Added Capacity"):
#             return ("Bridge Rehabilitation in " + df[name_col])
        
#         elif (df[flag_col] == "Project") & (df[desc_col] == "Facilities for Pedestrians and Bicycles"):
#             return (df[desc_col] + " in " + df[name_col])
        
#         elif (df[flag_col] == "Project") & (df[desc_col] == "Safety"):
#             return (df[desc_col] + " Improvements in " + df[name_col])
        
#         # elif (df[flag_col] == "Project") & (df[flag_col] == "Planning") & (df[title_col].str.contains("REGION CONSOLIDATED PLANNING GRANT")):
#         #     return ("Regional Planning Grant in " + df[name_col])
            
#         elif (df[flag_col] == "Project") & (df[desc_col] == "Planning "):
#             return "Project Planning in " + df[name_col]
            
#         elif (df[flag_col] == "Project") & (df[desc_col] == "Preliminary Engineering"):
#             return (df[desc_col] + " Projects in " + df[name_col])
        
#         elif (df[flag_col] == "Project") & (df[desc_col] == "Construction Engineering"):
#             return (df[desc_col] + " Projects in " + df[name_col])
        
#         elif (df[flag_col] == "Project") & (df[desc_col] == "4R - Restoration & Rehabilitation"):
#             return ("Road Restoration & Rehabilitation in " + df[name_col])
        
#         elif (df[flag_col] == "Project") & (df[desc_col] == "4R - Maintenance  Resurfacing"):
#             return ("Maintenance Resurfacing in " + df[name_col])
        
#         elif (df[flag_col] == "Project") & (df[desc_col] == "Bridge Replacement - Added Capacity") | (df[desc_col] == "Bridge Replacement - No Added Capacity") | (df[desc_col] == "Bridge New Construction") | (df[desc_col] == "Special Bridge"):
#             return ("Bridge Replacement in " + df[name_col])
        
#         elif (df[flag_col] == "Project") & (df[desc_col] == "Mitigation of Water Pollution due to Highway Runoff"):
#             return (df[desc_col] + " in " + df[name_col])
        
#         elif (df[flag_col] == "Project") & (df[desc_col] == "4R - Added Capacity"):
#             return ("Added Roadway Capacity in " + df[name_col])
        
#         elif (df[flag_col] == "Project") & (df[desc_col] == "4R - No Added Capacity"):
#             return ("Road Construction in " + df[name_col])
        
#         elif (df[flag_col] == "Project") & (df[desc_col] == "New  Construction Roadway"):
#             return ("New Construction Roadway in " + df[name_col])
        
#         elif (df[flag_col] == "Project") & (df[desc_col] == "Traffic Management/Engineering - HOV"):
#             return ("Traffic Management Project in " + df[name_col])
        
#         elif (df[flag_col] == "Project") & (df[desc_col] != "Other"):
#             return (df[desc_col] + " in " + df[name_col])
        
#         else:
#             return "" #(df[desc_col] + " in " + df[name_col])

#         return df


#     df['project_name_new2'] = df.apply(return_project_type, axis = 1)
    
#     #df.apply(func, axis=1)
    
#     return df

In [48]:
no_match_title = (utils.update_no_matched(no_match, 'project_type', 'improvement_type_description', 'implementing_agency'))

In [49]:
no_match_title.sample()

project_number recipient_project_number  \
492        0010203              0022000348S   

                                                                                                                                             project_title  \
492  OWP 22/23 SACRAMENTO REGION CONSOLIDATED PLANNING GRANT (CPG) PROJECT FOR SACRAMENTO AREA COUNCIL OF GOVERNMENTS (SACOG) OVERALL WORK PROGRAM (OWP) F   

    improvement_type_description implementing_agency county_name  \
492                    Planning           California   Statewide   

    project_method project_type    project_name_new  \
492                     Project   Project Statewide   

                  project_name_new2  
492  Project Planning in California

In [50]:
# remaining = (no_match_title>>filter(_.project_name_new2=='')).improvement_type_description.value_counts()
# remaining

### Remaining 
| Description | Count |
| --- | --- |
|Training | 69 |
| Other  |  16 |
| Right of Way | 13 |
|  Safety and Education for Peds/Bicyclist | 1 |
| Bridge Inspection and Bridge Related Training | 1 |
| Rail/Hwy Crossing    |  1 |
| Utilities    | 1 |
| Landscaping and Other Scenic Beautification   |    1 |
| Research   |  1 |


In [51]:
df>>filter(_.project_title.str.contains("CHOKEPOINT RELIEF"))

fmis_transaction_date program_code        program_code_description  \
576            2022-09-22         Y460  NATIONAL HWY FREIGHT PROG IIJA   
577            2022-09-22         Y460  NATIONAL HWY FREIGHT PROG IIJA   
578            2022-09-22         Y460  NATIONAL HWY FREIGHT PROG IIJA   

    project_number recipient_project_number  \
576        P057073              0715000076S   
577        P057073              0715000076S   
578        P057073              0715000076S   

                                                                                                                                             project_title  \
576  IN LOS ANGELES COUNTY. ROUTE 57/60 CONFLUENCE CHOKEPOINT RELIEF PROGRAM. RECONSTRUCT GRAND AVENUE OVERCROSSING. RECONSTRUCT NORTHBOUND SR-57 CONNECTO   
577  IN LOS ANGELES COUNTY. ROUTE 57/60 CONFLUENCE CHOKEPOINT RELIEF PROGRAM. RECONSTRUCT GRAND AVENUE OVERCROSSING. RECONSTRUCT NORTHBOUND SR-57 CONNECTO   
578  IN LOS ANGELES COUNTY. ROUTE 57/60 CONFLUENCE CHOKEPOINT RELIEF PROGRAM. RECONSTRUCT GRAND AVENUE OVERCROSSING. RECONSTRUCT NORTHBOUND SR-57 CONNECTO   

     county_code congressional_district  improvement_type  \
576        37.00           Cong Dist 39             11.00   
577        37.00           Cong Dist 39             17.00   
578        37.00           Cong Dist 39             42.00   

               improvement_type_description  obligations_amount  \
576  Bridge Replacement - No Added Capacity       72,969,500.00   
577                Construction Engineering        1,000,000.00   
578                                Training           30,500.00   

    summary_recipient_defined_text_field_1_value  implementing_agency_locode  \
576                                    L6065SCAG                    6,065.00   
577                                    L6065SCAG                    6,065.00   
578                                    L6065SCAG                    6,065.00   

                                          implementing_agency  district  \
576  Los Angeles County Metropolitan Transportation Authority      7.00   
577  Los Angeles County Metropolitan Transportation Authority      7.00   
578  Los Angeles County Metropolitan Transportation Authority      7.00   

            county_name                                             rtpa_name  \
576  Los Angeles County  Los Angeles County Metropolitan Transportation Auth.   
577  Los Angeles County  Los Angeles County Metropolitan Transportation Auth.   
578  Los Angeles County  Los Angeles County Metropolitan Transportation Auth.   

                                           mpo_name project_method  \
576  Southern California Association Of Governments      Construct   
577  Southern California Association Of Governments      Construct   
578  Southern California Association Of Governments      Construct   

    project_type  \
576      Project   
577      Project   
578      Project   

                                                                  project_name_new  
576  Construct Project in Los Angeles County Metropolitan Transportation Authority  
577  Construct Project in Los Angeles County Metropolitan Transportation Authority  
578  Construct Project in Los Angeles County Metropolitan Transportation Authority

In [52]:
no_match_title>>filter(_.improvement_type_description=='Training')

project_number recipient_project_number  \
578        P057073              0715000076S   
580        X099645              1000000424S   
3          6801946              0414000019S   
11         8801911              0416000001S   
18         000C524              0414000003S   
..             ...                      ...   
650        2152252              0819000045S   
653        2801153              0416000024S   
656        2801153              0416000024S   
661        4053001              0716000031S   
664        4053001              0716000031S   

                                                                                                                                             project_title  \
578  IN LOS ANGELES COUNTY. ROUTE 57/60 CONFLUENCE CHOKEPOINT RELIEF PROGRAM. RECONSTRUCT GRAND AVENUE OVERCROSSING. RECONSTRUCT NORTHBOUND SR-57 CONNECTO   
580   IN AND NEAR THE CITY OF MODESTO ON SR 132 NEAR NORTH DAKOTA AVE TO NEAR 6TH ST AND ON SR 99 FROM 0.5 M SOUTH OF TUOLUMNE RIVER BRIDGE TO 0.1 M NORTH   
3                                                                   ALAMEDA COUNTY AT SUNOL AND IN PLEASANTON AND DUBLIN PRECAST CONCRETE PAVEMENT AND HMA   
11                              ALAMEDA COUNTY IN SAN LEANDRO AT WASHINGTON AVENUE OVERCROSSING ROADWAY EXCAVATION, HMA, AND MODIFYING ELECTRICAL SYSTEMS.   
18                                                                     ALAMEDA AND CONTRA COSTA COUNTIES AT VARIOUS LOCATIONS CROSSWALK SAFETY ENHANCEMENT   
..                                                                                                                                                     ...   
650  IN RIVERSIDE COUNTY NEAR MORENO VALLEY ON INTERSTATE 215 FROM 0.2 MILE NORTH OF ALESSANDRO BOULEVARD OVERCROSSING TO 0.2 MILE SOUTH OF EUCALYPTUS AVE   
653        SANTA CLARA COUNTY IN AND NEAR SAN JOSE FROM S280-N101/101 SEPARATION TO ROUTE 17/280 SEPERATION MINOR CONCRETE, MGS RAILING, HMA AND HMA DIKE.   
656        SANTA CLARA COUNTY IN AND NEAR SAN JOSE FROM S280-N101/101 SEPARATION TO ROUTE 17/280 SEPERATION MINOR CONCRETE, MGS RAILING, HMA AND HMA DIKE.   
661  IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE   
664  IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE   

    improvement_type_description  \
578                     Training   
580                     Training   
3                       Training   
11                      Training   
18                      Training   
..                           ...   
650                     Training   
653                     Training   
656                     Training   
661                     Training   
664                     Training   

                                          implementing_agency  \
578  Los Angeles County Metropolitan Transportation Authority   
580                                                   Modesto   
3                                                  California   
11                                                 California   
18                                                 California   
..                                                        ...   
650                                                California   
653                                                California   
656                                                California   
661                                                California   
664                                                California   

            county_name project_method project_type  \
578  Los Angeles County      Construct      Project   
580   Stanislaus County                     Project   
3        Alameda County           Pave      Project   
11       Alameda County                     Project   
18            Statewide                     Project  

In [53]:
((utils.get_list_of_words((no_match_title>>filter(_.improvement_type_description=='Training')), "project_title")).value_counts()).iloc[0:40]

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


mile             64
county           50
route            35
near             31
north            25
road             25
south            25
los              23
san              22
west             22
02               20
angeles          19
overcrossing     18
avenue           16
boulevard        14
construct        13
santa            13
hma              13
locations        12
01               12
east             12
various          12
bridge           11
05               11
separation       10
undercrossing    10
miles            10
04                9
canyon            8
03                8
orange            7
city              7
street            6
counties          6
systems           6
install           6
replace           6
drainage          6
upgrade           6
safety            5
dtype: int64

In [54]:
# df>>filter(_.improvement_type_description=='Training')>>select(_.program_code_description, _.project_title, _.improvement_type_description,
#                                                               _.project_method, _.project_type, _.project_name_new)

In [55]:
## potential duplicates. different "descriptions 

df>>filter(_.project_title.str.contains("ERN COUNTY IN THE CITY OF BAKERSFIELD ON ROUTE 99 FROM 0.3 MILE SOUTH OF BELLE TERRACE OVERCROSSING TO 0.1 MILE NORTH OF ROUTE 58"))>>select(_.program_code_description, _.project_title, _.improvement_type_description,
                                                              _.project_method, _.project_type, _.project_name_new)

program_code_description  \
398  NATIONAL HIGHWAY PERF IIJA   
399  NATIONAL HIGHWAY PERF IIJA   
400  NATIONAL HIGHWAY PERF IIJA   
401  NATIONAL HIGHWAY PERF IIJA   
402  NATIONAL HIGHWAY PERF IIJA   
403  NATIONAL HIGHWAY PERF IIJA   

                                                                                                                                            project_title  \
398  KERN COUNTY IN THE CITY OF BAKERSFIELD ON ROUTE 99 FROM 0.3 MILE SOUTH OF BELLE TERRACE OVERCROSSING TO 0.1 MILE NORTH OF ROUTE 58; ALSO ON ROUTE 58   
399  KERN COUNTY IN THE CITY OF BAKERSFIELD ON ROUTE 99 FROM 0.3 MILE SOUTH OF BELLE TERRACE OVERCROSSING TO 0.1 MILE NORTH OF ROUTE 58; ALSO ON ROUTE 58   
400  KERN COUNTY IN THE CITY OF BAKERSFIELD ON ROUTE 99 FROM 0.3 MILE SOUTH OF BELLE TERRACE OVERCROSSING TO 0.1 MILE NORTH OF ROUTE 58; ALSO ON ROUTE 58   
401  KERN COUNTY IN THE CITY OF BAKERSFIELD ON ROUTE 99 FROM 0.3 MILE SOUTH OF BELLE TERRACE OVERCROSSING TO 0.1 MILE NORTH OF ROUTE 58; ALSO ON ROUTE 58   
402  KERN COUNTY IN THE CITY OF BAKERSFIELD ON ROUTE 99 FROM 0.3 MILE SOUTH OF BELLE TERRACE OVERCROSSING TO 0.1 MILE NORTH OF ROUTE 58; ALSO ON ROUTE 58   
403  KERN COUNTY IN THE CITY OF BAKERSFIELD ON ROUTE 99 FROM 0.3 MILE SOUTH OF BELLE TERRACE OVERCROSSING TO 0.1 MILE NORTH OF ROUTE 58; ALSO ON ROUTE 58   

               improvement_type_description project_method project_type  \
398                  4R - No Added Capacity                     Project   
399                  4R - No Added Capacity                     Project   
400  Bridge Replacement - No Added Capacity                     Project   
401  Bridge Replacement - No Added Capacity                     Project   
402                                Training                     Project   
403                                Training                     Project   

            project_name_new  
398   Project in Kern County  
399   Project in Kern County  
400   Project in Kern County  
401   Project in Kern County  
402   Project in Kern County  
403   Project in Kern County

In [56]:
no_match_title >>filter(_.project_title.str.contains('BRIDGE'))>>select(_.project_title, _.improvement_type_description,
                                                              _.project_method, _.project_type, _.project_name_new, _.project_name_new2)>>filter(_.project_name_new2=="")

Empty DataFrame
Columns: [project_title, improvement_type_description, project_method, project_type, project_name_new, project_name_new2]
Index: []

### Try with one element of project

- projects listed can have multiple entires bases on the phases they have. currently, if a project has multiple entries, the `improvement_type_description` has different description phrases, leading the title function to create a title that does not accuratley reflect the program
- what we want to do is apply the function to one entry and the use that accross the multiple projects

In [57]:
## How many projects have multiple funding componenets?
## a sample of those under the no_match flag with at least two entires in system

(no_match
 >>group_by(_.recipient_project_number, _.project_number, _.implementing_agency, _.project_title)
 >>summarize(n_descriptions = _.improvement_type_description.nunique())
 >>filter(_.n_descriptions>1)
 >>arrange(_.n_descriptions, _.n_descriptions)
)

recipient_project_number project_number implementing_agency  \
23               0113000123S        P020198          California   
27               0117000018S        P020199          California   
30               0213000002S        0059180          California   
31               0216000044S        0058644          California   
35               0300020624S        P020179          California   
..                       ...            ...                 ...   
132              0615000046S        0055155          California   
154              0716000064S        0053074          California   
187              0900000030S        P395348          California   
192              1013000266S        S108057          California   
223              1218000034S        X059084          California   

                                                                                                                                             project_title  \
23          IN MENDOCINO COUNTY NEAR UKIAH FROM 0.3 MILE WEST OF RUSSIAN RIVER BRIDGE AND OVERHEAD TO 0.4 MILE EAST OF COUNTY ROAD 144 REPLACE TWO BRIDGES   
27   IN LAKE COUNTY NEAR CLEAR LAKE OAKS AT BERYL WAY THIS PROJECT PROPOSES TO INSTALL A COMBINATION OF TWO ALTERNATIVE IN-LINE TERMINAL SYSTEMS, A RAIL E   
30   IN SISKIYOU COUNTY, AT VARIOUS LOCATIONS FROM 0.2 MILE NORTH OF REAM AVENUE OVERCROSSING TO 0.2 MILE SOUTH OF HILT ROAD OVERCROSSING. ENHANCE TRAFFIC   
31             IN TEHAMA COUNTY NEAR RED BLUFF FROM NINE MILE HILL OVERCROSSING TO 0.1 MILE NORTH OF BOWMAN ROAD OVERCROSSING. IMPROVE CLEAR RECOVERY ZONE   
35   NEAR LAKE WILDWOOD FROM 0.4 MILE EAST OF MCGANNERY LANE TO YUBA/NEVADA COUNTY LINE; ALSO IN NEVADA COUNTY, FROM YUBA/NEVADA COUNTY LINE TO MOONEY FLA   
..                                                                                                                                                     ...   
132   FRESNO COUNTY NEAR MENDOTA FROM 1.2 MILES NORTH OF HUDSON AVENUE OVERCROSSING TO 0.4 MILE SOUTH OF MANNING AVENUE OVERCROSSING. REPLACE  TUMEY GULCH   
154   NEAR CASTAIC, FROM LAKE HUGHES ROAD UNDERCROSSING TO 1.9 MILES NORTH OF TEMPLIN HIGHWAY. REPLACE APPROACH SLABS, UPGRADE DRAINAGE SYSTEMS, UPGRADING   
187  IN INYO COUNTY NEAR OLANCHA AND CARTAGO FROM 1.4 MILES SOUTH OF LOS ANGELES AQUEDUCT BRIDGE TO 0.2 MILE SOUTH OF ASH CREEK BRIDGE. CONSTRUCT 4 LANE E   
192  TUOLUMNE COUNTY IN AND NEAR LONG BARN FROM 0.1 MILE WEST OF LONG BARN ROAD TO 2.3 MILES EAST OF COW CREEK ROAD COLD PLANE, HMA, DRAINAGE REPLACEMENT,   
223  CONST BRIDGES, RW, SW, JPCP, HMA PVMT, DRAINAGE AND ELECT SYSTEMS. ORANGE COUNTY IN IRVINE, SANTA ANA, AND TUSTIN ON ROUTE 5 AT NEWPORT AVENUE UNDERC   

     n_descriptions  
23                2  
27                2  
30                2  
31                2  
35                2  
..              ...  
132               4  
154               4  
187               4  
192               4  
223               4  

[138 rows x 5 columns]

* of the 133 projects wiht unique project IDs, 60 have more than 1 entry
* using all unique regardless of numbers 

In [58]:
df_copy = (df>>filter(_.project_type==('Project')))

In [59]:
len(df>>filter(_.project_type==('Project'))
 >>group_by(_.fmis_transaction_date, _.program_code, _.program_code_description, _.project_number,
                                                   _.recipient_project_number, _.project_title, _.county_code, _.congressional_district,
                                                   _.summary_recipient_defined_text_field_1_value, _.implementing_agency_locode,
                                                   _.implementing_agency, _.district, _.county_name, _.rtpa_name, _.mpo_name)
 >>summarize(n_descriptions = _.improvement_type_description.nunique())>>filter(_.n_descriptions>1)>>arrange(_.n_descriptions, _.n_descriptions))

59

In [60]:
len(df>>filter(_.project_type==('Project'))
 >>group_by(_.fmis_transaction_date, _.program_code, _.program_code_description, _.project_number,
                                                   _.recipient_project_number, _.project_title, _.county_code, _.congressional_district,
                                                   _.summary_recipient_defined_text_field_1_value, _.implementing_agency_locode,
                                                   _.implementing_agency, _.district, _.county_name, _.rtpa_name, _.mpo_name)
 >>summarize(n_descriptions = _.improvement_type_description.nunique()))

125

In [61]:
proj_unique = (df>>filter(_.project_type==('Project'))
#  >>group_by(_.fmis_transaction_date, _.program_code, _.program_code_description, _.project_number,
#                                                    _.recipient_project_number, _.project_title, _.county_code, _.congressional_district,
#                                                    _.summary_recipient_defined_text_field_1_value, _.implementing_agency_locode,
#                                                    _.implementing_agency, _.district, _.county_name, _.rtpa_name, _.mpo_name)
# >>arrange(_.project_number, _.obligations_amount)
              )

#### Use .max() to get the max value for obligation amount (the main portion of the funding)

In [62]:
#proj_unique.drop_duplicates('project_number', keep='last')
proj_unique = proj_unique.groupby(['fmis_transaction_date','program_code','project_number','recipient_project_number','project_title',
                    'implementing_agency', 'district', 'county_name'])['obligations_amount'].max().reset_index()

In [63]:
merge_cols = ['fmis_transaction_date', 'program_code','project_number','recipient_project_number','project_title',
                    'implementing_agency', 'county_name', 'district', 'obligations_amount']

#### Merge to get full details

In [64]:
proj_unique = (pd.merge(proj_unique, df_copy, how='left', on=merge_cols))

In [65]:
proj_unique.sample(4)

fmis_transaction_date program_code project_number  \
79             2022-05-20         YS30        P133016   
11             2022-02-16         Y233        5917084   
73             2022-05-16         YS32        P091150   
203            2022-09-13         Y240        0154210   

    recipient_project_number  \
79               1214000077S   
11               0314000112L   
73               0719000078S   
203              1119000008S   

                                                                                                                                             project_title  \
79   IN ORANGE COUNTY, IN AND NEAR LAGUNA BEACH AND IRVINE, FROM NORHT OF EL TORO ROAD TO RTE. 241. FLATTEN STEEP EMBANKMENTS, REPLACE AND UPGRADE GUARDRA   
11           DOG BAR ROAD AT BEAR RIVER ~0.5 MILES SOUTH EAST OF MAGNOLIA ROAD AT COUNTY LINE. BR. # 17C0031 REPLACE 1 LANE BRIDGE WITH 2 LANE BRIDGE (TC)   
73   IN LOS ANGELES COUNTY, IN COMPTON FROM 0,2 MILE WEST TO COMPTON CREEK BRIDGE AND OVERHEAD.  THIS PROJECT WILL UPGRADE EXISTING HIGHWAY SAFETY FEATURE   
203  IN SAN DIEGO COUNTY ON INTERSTATE 15 IN AND NEAR ESCONDIDO, FROM 0.2 MILE SOUTH OF ROUTE 15/78 SEPARATIONTO 0.5 MILE NORTH OF DEER SPRINGS ROAD OVERC   

    implementing_agency  district         county_name  obligations_amount  \
79           California     12.00       Orange County          580,775.17   
11        Nevada County      3.00       Nevada County           50,000.00   
73           California      7.00  Los Angeles County          662,053.96   
203          California     11.00    San Diego County       47,312,900.00   

           program_code_description  county_code congressional_district  \
79     HIGHWAY SAFETY IMP PROG IIJA        59.00           Cong Dist 45   
11      STBG IIJA OFF-SYSTEM BRIDGE        57.00            Cong Dist 1   
73      SEC 164 PENALTIES HSIP IIJA        37.00           Cong Dist 44   
203  SURFAC TRNSP BLK GRTS-FLX IIJA        73.00           Cong Dist 50   

     improvement_type       improvement_type_description  \
79              21.00                             Safety   
11              16.00                       Right of Way   
73              21.00                             Safety   
203              6.00  4R - Restoration & Rehabilitation   

    summary_recipient_defined_text_field_1_value  implementing_agency_locode  \
79                                     S    SCAG                         NaN   
11                                  L5917NON-MPO                    5,917.00   
73                                     S    SCAG                         NaN   
203                                  S    SANDAG                         NaN   

                                   rtpa_name mpo_name project_method  \
79                                       NaN      NaN        Upgrade   
11   Nevada County Transportation Commission  NON-MPO        Replace   
73                                       NaN      NaN        Upgrade   
203                                      NaN      NaN                  

    project_type                       project_name_new  
79       Project       Upgrade Project in Orange County  
11       Project       Replace Project in Nevada County  
73       Project  Upgrade Project in Los Angeles County  
203      Project            Project in San Diego County

#### run with update_no_matched function

In [66]:
proj_unique_new = utils.update_no_matched(proj_unique, 'project_type', 'improvement_type_description', 'implementing_agency')

In [67]:
proj_unique_new.sample(5)

fmis_transaction_date program_code project_number  \
207            2022-09-14         Y003        5256021   
122            2022-07-01         YS30        5007090   
179            2022-09-07         YS30        P004158   
2              2022-02-10         Y230        6480013   
87             2022-06-07         YS30        5164035   

    recipient_project_number  \
207              1022000098L   
122              0520000006L   
179              1016000139S   
2                0417000451L   
87               0721000199L   

                                                                                                                                             project_title  \
207  IN LIVINGSTON, CONSTRUCT PHASE 2 MAX FOSTER MULTIUSE PATH (ALONG DWIGHT WAY, BETWEEN NORTH TRIGGER LN & TEHAMA DR,) CONSTRUCT A CONCRETE MULTIUSE PAT   
122  INTERSECTIONS OF BATH ST/SOLA ST, BATH ST/VICTORIA ST, CANON PERDIDO ST/NOPAL ST, AND OLD COAST HIGHWAY/SALINAS ST IN THE CITY OF SANTA BARBARA. ADDI   
179                    SAN JOAQUIN COUNTY NEAR HOLT FROM MIDDGE RIVER TO 0.4 MILE EAST OF MIDDLE RIVER SAN JOAQUIN MIDDLE RIVER BRIDGE ROADWAY REALIGNMENT   
2    COUNTY WIDE- APPROXIMATELY 300 PUBLIC SCHOOL ALAMEDA COUNTY SAFE ROUTS TO SCHOOL PROGRAM OUTREACH AND EDUCATION. WORK PROGRAM IS FOCUSED ON SUPPORTIN   
87   VARIOUS UNCONTROLLED CROSSWALKS THROUGHOUT THE CITY OF INGLEWOOD, INCLUDING CENTINELA AVE AT MARKET ST, MANCHESTER BLVD AT CEDAR AVE, ON REGENT ST BE   

                          implementing_agency  district           county_name  \
207                                Livingston     10.00         Merced County   
122                             Santa Barbara      5.00  Santa Barbara County   
179                                California     10.00    San Joaquin County   
2    Alameda County Transportation Commission      4.00        Alameda County   
87                                  Inglewood      7.00    Los Angeles County   

     obligations_amount        program_code_description  county_code  \
207          373,016.00  PROJ TO REDUCE PM 2.5 EMI IIJA        47.00   
122        1,120,563.00    HIGHWAY SAFETY IMP PROG IIJA        83.00   
179        6,066,000.00    HIGHWAY SAFETY IMP PROG IIJA        77.00   
2          1,500,000.00       STBG-URBANIZED >200K IIJA         1.00   
87           250,000.00    HIGHWAY SAFETY IMP PROG IIJA        37.00   

    congressional_district  improvement_type  \
207           Cong Dist 16             28.00   
122           Cong Dist 24             21.00   
179            Cong Dist 9             21.00   
2             Cong Dist 13             38.00   
87            Cong Dist 43             21.00   

                 improvement_type_description  \
207   Facilities for Pedestrians and Bicycles   
122                                    Safety   
179                                    Safety   
2    Safety and Education for Peds/Bicyclists   
87                                     Safety   

    summary_recipient_defined_text_field_1_value  implementing_agency_locode  \
207                                    L5256MCAG                    5,256.00   
122                                   L5007SBCAG                    5,007.00   
179                                   S    SJCOG                         NaN   
2                                       L6480MTC                    6,480.00   
87                                     L5164SCAG                    5,164.00   

                                                rtpa_name  \
207              Merced County Association of Governments   
122       Santa Barbara County Association of Governments   
179                                                   NaN   
2                                                NON-RTPA   
87   Los Angeles County Metropolitan Transportation Auth.   

                                            mpo_name project_method  \
207          Merced County Association Of Goverments      Construct   
122  Santa Barbara County

In [68]:
len(proj_unique_new>>filter(_.project_name_new2==""))

6

In [69]:
(proj_unique_new>>filter(_.project_name_new2=="")>>select(_.project_number, _.recipient_project_number, _.project_title, _.improvement_type_description, _.implementing_agency, _.county_name,
                                                            _.project_method, _.project_type, _.project_name_new, _.project_name_new2))

project_number recipient_project_number  \
16         5108202              0722000055L   
81         6084279              0422000413L   
95         0052029              1216000044S   
102        4053001              0716000031S   
103        P014105              0716000029S   
134        6085086              0322000305L   

                                                                                                                                             project_title  \
16   DOWNTOWN LONG BEACH (BOUNDED BY MAGNOLIA AVE TO THE WEST, OCEAN BLVD TO THE SOUTH, 7TH STREET TO THE NORTH, AND ALAMITOS AVE TO THE EAST). NON-INFRAS   
81                                                                             SF BAY AREA: REGIONWIDE TECHNICAL ASSISTANCE MOBILITY HUB PILOT PROGRAM(TC)   
95   ORANGE COUNTY, IN ANAHEIM FROM SANTA ANA STREET OVERCROSSING TO 0.5 MILE NORTH OF LA PALMA AVENUE OVERCROSSING PLANTING, IRRIGATION, ROCK BLANKET, MI   
102  IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE   
103   IN AND NEAR SANTA CLARITA, FROM CANYON PARK BOULEVARD TO SPRING CANYON ROAD UNDERCROSSING. MINOR CONCRETE BEYOND GORE, SLOPE PAVE BENEATH ABUTMENTS,   
134  SACOG REGION - IN SACRAMENTO, SUTTER, YOLO AND YUBA COUNTIES CONDUCT NECESSARY PLANNING, STATE AND FEDERAL PROGRAMMING, AND MONITORING ACTIVITIES, IN   

    improvement_type_description                     implementing_agency  \
16                         Other                              Long Beach   
81                         Other  Metropolitan Transportation Commission   
95                         Other                              California   
102                        Other                              California   
103                        Other                              California   
134                        Other  Sacramento Area Council of Governments   

            county_name project_method project_type  \
16   Los Angeles County                     Project   
81         Multi-County                     Project   
95        Orange County                     Project   
102  Los Angeles County                     Project   
103  Los Angeles County           Pave      Project   
134   Sacramento County                     Project   

                                       project_name_new project_name_new2  
16                                Project in Long Beach                    
81    Project in Metropolitan Transportation Commission                    
95                             Project in Orange County                    
102                       Project in Los Angeles County                    
103                  Pave Project in Los Angeles County                    
134   Project in Sacramento Area Council of Governments

In [70]:
remaining = (proj_unique_new>>filter(_.project_name_new2=='')).improvement_type_description.value_counts()
remaining

Other    6
Name: improvement_type_description, dtype: int64

In [71]:
(proj_unique_new>>filter(_.improvement_type_description=='Other'))>>select(_.project_title, _.implementing_agency,
                                                                          _.project_method,_.project_type, _.project_name_new,_.project_name_new2)



project_title  \
16   DOWNTOWN LONG BEACH (BOUNDED BY MAGNOLIA AVE TO THE WEST, OCEAN BLVD TO THE SOUTH, 7TH STREET TO THE NORTH, AND ALAMITOS AVE TO THE EAST). NON-INFRAS   
81                                                                             SF BAY AREA: REGIONWIDE TECHNICAL ASSISTANCE MOBILITY HUB PILOT PROGRAM(TC)   
95   ORANGE COUNTY, IN ANAHEIM FROM SANTA ANA STREET OVERCROSSING TO 0.5 MILE NORTH OF LA PALMA AVENUE OVERCROSSING PLANTING, IRRIGATION, ROCK BLANKET, MI   
102  IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE   
103   IN AND NEAR SANTA CLARITA, FROM CANYON PARK BOULEVARD TO SPRING CANYON ROAD UNDERCROSSING. MINOR CONCRETE BEYOND GORE, SLOPE PAVE BENEATH ABUTMENTS,   
134  SACOG REGION - IN SACRAMENTO, SUTTER, YOLO AND YUBA COUNTIES CONDUCT NECESSARY PLANNING, STATE AND FEDERAL PROGRAMMING, AND MONITORING ACTIVITIES, IN   

                        implementing_agency project_method project_type  \
16                               Long Beach                     Project   
81   Metropolitan Transportation Commission                     Project   
95                               California                     Project   
102                              California                     Project   
103                              California           Pave      Project   
134  Sacramento Area Council of Governments                     Project   

                                       project_name_new project_name_new2  
16                                Project in Long Beach                    
81    Project in Metropolitan Transportation Commission                    
95                             Project in Orange County                    
102                       Project in Los Angeles County                    
103                  Pave Project in Los Angeles County                    
134   Project in Sacramento Area Council of Governments